In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Mar 21 22:42:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:01:01.0 Off |                    0 |
| N/A   31C    P0    36W / 250W |  19160MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:01:02.0 Off |                    0 |
| N/A   

In [2]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = '1'
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from enum import Enum
import imageio
import hashlib
import copy
import time

%matplotlib inline
plt.style.use('seaborn-whitegrid')
%config InlineBackend.figure_format='retina'

dtype = 'float32'
tf.keras.backend.set_floatx(dtype)

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

2022-03-21 22:42:39.086469: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-21 22:42:39.087319: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-21 22:42:39.146963: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-21 22:42:39.147591: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-21 22:42:39.147924: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [3]:
################################################################################
# DATASETS
################################################################################


def get_dataset_sample(X, y, fraction, seed=None):
    if seed is not None:
        np.random.seed(seed)  # Set random seed
    selection = np.random.choice([True, False], len(X), p=[fraction, 1 - fraction])
    if seed is not None:
        np.random.seed()  # Unset random seed
    X_sampled = X[selection]
    y_sampled = y[selection]
    return X_sampled, y_sampled


class Dataset:
    def __init__(self, X_train, y_train, X_test, y_test, shape, shape_flattened, fraction, vision=True, standardize=True):
        if fraction is not None:
            X_train, y_train = get_dataset_sample(X_train, y_train, fraction, seed=42)
            X_test, y_test = get_dataset_sample(X_test, y_test, fraction, seed=42)
        
        X_train = X_train.astype(dtype)
        y_train = y_train.astype(dtype)
        X_test = X_test.astype(dtype)
        y_test = y_test.astype(dtype)

        if vision:
            X_train = X_train / 255.0
            X_test = X_test / 255.0

        X_train = np.reshape(X_train, shape_flattened)
        X_test = np.reshape(X_test, shape_flattened)

        X = np.concatenate((X_train, X_test), axis=0)
        y = np.concatenate((y_train, y_test), axis=0)

        if standardize:
            from sklearn.preprocessing import StandardScaler

            scaler = StandardScaler()
            scaler.fit(X_train)  # Scaling each feature independently

            X_norm = scaler.transform(X)
            X_train_norm = scaler.transform(X_train)
            X_test_norm = scaler.transform(X_test)
        else:
            X_norm = X.copy()
            X_train_norm = X_train.copy()
            X_test_norm = X_test.copy()

        X_norm = np.reshape(X_norm, shape)
        X_train_norm = np.reshape(X_train_norm, shape)
        X_test_norm = np.reshape(X_test_norm, shape)

        del X, X_train, X_test

        self.X_norm = X_norm
        self.y = y
        self.X_train_norm = X_train_norm
        self.y_train = y_train
        self.X_test_norm = X_test_norm
        self.y_test = y_test


def get_cifar_10_dataset(fraction=None):
    cifar10 = tf.keras.datasets.cifar10
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    shape = (-1, 32, 32, 3)
    shape_flattened = (-1, 3072)  # Scaling each feature independently
    return Dataset(X_train, y_train, X_test, y_test, shape=shape, shape_flattened=shape_flattened, fraction=fraction)


def get_cifar_100_dataset(fraction=None):
    cifar100 = tf.keras.datasets.cifar100
    (X_train, y_train), (X_test, y_test) = cifar100.load_data()
    shape = (-1, 32, 32, 3)
    shape_flattened = (-1, 3072)  # Scaling each feature independently
    return Dataset(X_train, y_train, X_test, y_test, shape=shape, shape_flattened=shape_flattened, fraction=fraction)


def get_svhn_dataset(fraction=None):
    from urllib.request import urlretrieve
    from scipy import io

    train_filename, _ = urlretrieve('http://ufldl.stanford.edu/housenumbers/train_32x32.mat')
    test_filename, _ = urlretrieve('http://ufldl.stanford.edu/housenumbers/test_32x32.mat')

    X_train = io.loadmat(train_filename, variable_names='X').get('X')
    y_train = io.loadmat(train_filename, variable_names='y').get('y')
    X_test = io.loadmat(test_filename, variable_names='X').get('X')
    y_test = io.loadmat(test_filename, variable_names='y').get('y')

    X_train = np.moveaxis(X_train, -1, 0)
    y_train -= 1
    X_test = np.moveaxis(X_test, -1, 0)
    y_test -= 1

    shape = (-1, 32, 32, 3)
    shape_flattened = (-1, 3072)  # Scaling each feature independently
    return Dataset(X_train, y_train, X_test, y_test, shape=shape, shape_flattened=shape_flattened, fraction=fraction)


def get_tiny_imagenet_dataset(fraction=None):
    """
    Original source: https://github.com/sonugiri1043/Train_ResNet_On_Tiny_ImageNet/blob/master/Train_ResNet_On_Tiny_ImageNet.ipynb
    Original author: sonugiri1043@gmail.com
    """

    if not os.path.isdir('IMagenet'):
        ! git clone https://github.com/seshuad/IMagenet

    print("Processing the downloaded dataset...")

    path = 'IMagenet/tiny-imagenet-200/'

    id_dict = {}
    for i, line in enumerate(open(path + 'wnids.txt', 'r')):
        id_dict[line.replace('\n', '')] = i

    train_data = list()
    test_data = list()
    train_labels = list()
    test_labels = list()

    for key, value in id_dict.items():
        train_data += [imageio.imread(path + 'train/{}/images/{}_{}.JPEG'.format(key, key, str(i)), pilmode='RGB') for i in range(500)]
        train_labels_ = np.array([[0]*200]*500)
        train_labels_[:, value] = 1
        train_labels += train_labels_.tolist()

    for line in open(path + 'val/val_annotations.txt'):
        img_name, class_id = line.split('\t')[:2]
        test_data.append(imageio.imread(path + 'val/images/{}'.format(img_name), pilmode='RGB'))
        test_labels_ = np.array([[0]*200])
        test_labels_[0, id_dict[class_id]] = 1
        test_labels += test_labels_.tolist()

    X_train = np.array(train_data)
    y_train = np.argmax(np.array(train_labels), axis=1)
    X_test = np.array(test_data)
    y_test = np.argmax(np.array(test_labels), axis=1)

    shape = (-1, 64, 64, 3)
    shape_flattened = (-1, 12288)  # Scaling each feature independently
    return Dataset(X_train, y_train, X_test, y_test, shape=shape, shape_flattened=shape_flattened, fraction=fraction)


def get_mnist_dataset(fraction=None):
    mnist = tf.keras.datasets.mnist
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    shape = (-1, 28, 28, 1)
    shape_flattened = (-1, 1)  # Scaling all features together
    return Dataset(X_train, y_train, X_test, y_test, shape=shape, shape_flattened=shape_flattened, fraction=fraction)


def get_fashion_mnist_dataset(fraction=None):
    fashion_mnist = tf.keras.datasets.fashion_mnist
    (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
    shape = (-1, 28, 28, 1)
    shape_flattened = (-1, 1)  # Scaling all features together
    return Dataset(X_train, y_train, X_test, y_test, shape=shape, shape_flattened=shape_flattened, fraction=fraction)


def get_fifteen_puzzle_dataset(path=None, fraction=None):
    from sklearn.model_selection import train_test_split

    if path is None:
        from google.colab import drive
        drive.mount('/content/gdrive')
        path = 'gdrive/MyDrive/15-costs-v3.csv'
    costs = pd.read_csv(path)

    X_raw = costs.iloc[:,:-1].values
    y = costs['cost'].values
    X = np.apply_along_axis(lambda x: np.eye(16)[x].ravel(), 1, X_raw)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    del X, X_raw, y

    shape = (-1, 256)
    shape_flattened = (-1, 256)  # Scaling all features together
    return Dataset(X_train, y_train, X_test, y_test, shape=shape, shape_flattened=shape_flattened, vision=False, fraction=fraction)


################################################################################
# REGULARIZERS
################################################################################


class Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self):
        self.n_new_neurons = 0
        self.scaling_tensor = None
        self.set_regularization_penalty(0.)
        self.set_regularization_method(None)

    def __call__(self, x):
        if self.regularization_method is None or self.regularization_penalty == 0:
            return 0
        elif self.regularization_method == 'weighted_l1':
            return self.weighted_l1(x)
        elif self.regularization_method == 'weighted_l1_reordered':
            return self.weighted_l1_reordered(x)
        elif self.regularization_method == 'group_sparsity':
            return self.group_sparsity(x)
        elif self.regularization_method == 'l1':
            return self.l1(x)
        else:
            raise NotImplementedError(f"Unknown regularization method {self.regularization_method}")
    
    def weighted_l1(self, x):
        # I.e. for a parameter matrix of 4 input and 10 output neurons:
        #
        # [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]
        #
        # the scaling tensor, as well as the resulting weighted values, could be:
        #
        # [[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        #  [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        #  [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        #  [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]]
        #
        # Therefore every additional output neuron is regularized more.

        scaling_tensor = tf.cumsum(tf.constant(self.regularization_penalty, shape=x.shape, dtype=dtype), axis=-1)
        weighted_values = scaling_tensor * tf.abs(x)
        return tf.reduce_sum(weighted_values)
    
    def weighted_l1_reordered(self, x):
        # I.e. for a parameter matrix of 4 input and 10 output neurons:
        #
        # [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]
        #
        # the scaling tensor, as well as the resulting weighted values, could be:
        #
        # [[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        #  [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        #  [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
        #  [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]]
        #
        # Therefore every additional output neuron is regularized more.

        if self.update_scaling_tensor:
            scaling_tensor_raw = tf.cumsum(tf.constant(self.regularization_penalty, shape=x.shape, dtype=dtype), axis=-1)

            scaling_tensor_old_neurons = scaling_tensor_raw[:, :-self.n_new_neurons]
            scaling_tensor_new_neurons = scaling_tensor_raw[:, -self.n_new_neurons:]
            scaling_tensor_old_neurons_shuffled = tf.transpose(tf.random.shuffle(tf.transpose(scaling_tensor_old_neurons)))
            self.scaling_tensor = tf.concat([scaling_tensor_old_neurons_shuffled, scaling_tensor_new_neurons], axis=-1)
            self.update_scaling_tensor = False

        weighted_values = self.scaling_tensor * tf.abs(x)
        return tf.reduce_sum(weighted_values)
    
    def group_sparsity(self, x):
        # I.e. for a parameter matrix of 3 input and 5 output neurons:
        #
        # [[1., 1., 1., 1., 1.],
        #  [1., 2., 2., 1., 2.],
        #  [2., 2., 3., 1., 3.]]
        #
        # The resulting vector of group norms is [2., 2., 3., 1., 3.], therefore for
        # every output neuron, its incoming connections form a group.

        group_norms = tf.norm(x, ord=2, axis=0)
        # assert group_norms.shape[0] == x.shape[1]
        return self.regularization_penalty * tf.reduce_sum(group_norms)
    
    def l1(self, x):
        weighted_values = self.regularization_penalty * tf.abs(x)
        return tf.reduce_sum(weighted_values)
    
    def prune(self):
        self.n_new_neurons = 0
        if self.regularization_method == 'weighted_l1_reordered':
            self.update_scaling_tensor = True
    
    def grow(self, n_new_neurons):
        self.n_new_neurons = n_new_neurons
        if self.regularization_method == 'weighted_l1_reordered':
            self.update_scaling_tensor = True
    
    def set_regularization_penalty(self, regularization_penalty):
        self.regularization_penalty = regularization_penalty
    
    def set_regularization_method(self, regularization_method):
        self.regularization_method = regularization_method
        if self.regularization_method == 'weighted_l1_reordered':
            self.update_scaling_tensor = True
        else:
            self.update_scaling_tensor = None

    def get_config(self):
        return {'regularization_penalty': float(self.regularization_penalty)}


################################################################################
# LAYERS
################################################################################


class DASLayer(tf.keras.layers.Layer):
    def __init__(self, input_shape):
        super().__init__()

        self._input_shape = input_shape


class Dense(DASLayer):
    def __init__(self, units, activation, kernel_initializer='glorot_uniform', 
                 bias_initializer='zeros', input_shape=None, fixed_size=False,
                 regularizer=None):
        super().__init__(input_shape)

        self.units = units
        self.activation = activation
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        self.fixed_size = fixed_size
        
        self.A = tf.keras.activations.get(activation)
        self.W_init = tf.keras.initializers.get(kernel_initializer)
        self.b_init = tf.keras.initializers.get(bias_initializer)
        if regularizer is not None:
            self.regularizer = regularizer
        else:
            self.regularizer = Regularizer()
    
    # def copy(self):
    #     input_shape = self._input_shape
    #     regularizer_copy = self.regularizer.copy()
    #     layer_copy = Dense(self.units, self.activation, self.kernel_initializer, 
    #                        self.bias_initializer, input_shape, self.fixed_size, 
    #                        regularizer_copy)
    #     return layer_copy
    
    def build(self, input_shape):
        input_units = input_shape[-1]

        self.W = tf.Variable(
            name='W',
            initial_value=self.W_init(shape=(input_units, self.units), dtype=dtype),
            trainable=True)
        
        self.b = tf.Variable(
            name='b',
            initial_value=self.b_init(shape=(self.units,), dtype=dtype),
            trainable=True)
        
        self.add_regularizer_loss()
    
    def call(self, inputs, training=None):
        return self.A(tf.matmul(inputs, self.W) + self.b)
    
    def add_regularizer_loss(self):
        self.add_loss(lambda: self.regularizer(tf.concat([self.W, tf.reshape(self.b, (1, -1))], axis=0)))

    def get_size(self):
        return self.W.shape[0], self.W.shape[1]
    
    def prune(self, threshold, active_input_units_indices):
        # Remove connections from pruned units in previous layer
        new_W = tf.gather(self.W.value(), active_input_units_indices, axis=0)

        if self.fixed_size:
            active_output_neurons_indices = list(range(new_W.shape[1]))
        else:
            # Prune units in this layer
            weights_with_biases = tf.concat([new_W, tf.reshape(self.b.value(), (1, -1))], axis=0)
            neurons_are_active = tf.math.reduce_max(tf.abs(weights_with_biases), axis=0) >= threshold
            active_output_neurons_indices = tf.reshape(tf.where(neurons_are_active), (-1,))
            
            new_W = tf.gather(new_W, active_output_neurons_indices, axis=1)
            new_b = tf.gather(self.b.value(), active_output_neurons_indices, axis=0)

            self.b = tf.Variable(name='b', initial_value=new_b, trainable=True)

        self.W = tf.Variable(name='W', initial_value=new_W, trainable=True)

        self.regularizer.prune()
        return active_output_neurons_indices
    
    def grow(self, n_new_input_units, percentage, min_new_units, scaling_factor):
        if n_new_input_units > 0:
            # Add connections to grown units in previous layer
            W_growth = self.W_init(shape=(self.W.shape[0] + n_new_input_units, self.W.shape[1]), dtype=dtype)[-n_new_input_units:, :] * scaling_factor  # TODO is it better to be multiplying here by scaling_factor? It does help with not increasing the max weights of existing neurons when new neurons are added.
            new_W = tf.concat([self.W.value(), W_growth], axis=0)
        else:
            new_W = self.W.value()

        if self.fixed_size:
            n_new_output_units = 0
        else:
            # Grow new units in this layer
            n_new_output_units = max(min_new_units, int(new_W.shape[1] * percentage))
            if n_new_output_units > 0:
                W_growth = self.W_init(shape=(new_W.shape[0], new_W.shape[1] + n_new_output_units), dtype=dtype)[:, -n_new_output_units:] * scaling_factor
                b_growth = self.b_init(shape=(n_new_output_units,), dtype=dtype)  # TODO for all possible bias initializers to work properly, the whole bias vector should be initialized at once
                new_W = tf.concat([new_W, W_growth], axis=1)
                new_b = tf.concat([self.b.value(), b_growth], axis=0)

                self.b = tf.Variable(name='b', initial_value=new_b, trainable=True)

        self.W = tf.Variable(name='W', initial_value=new_W, trainable=True)

        self.regularizer.grow(n_new_output_units)
        return n_new_output_units
    
    def mutate(self, mutation_strength):
        self.W.assign_add(tf.random.normal(self.W.shape, mean=0.0, stddev=mutation_strength))
        self.b.assign_add(tf.random.normal(self.b.shape, mean=0.0, stddev=mutation_strength))
    
    def set_regularization_penalty(self, regularization_penalty):
        if not self.fixed_size:
            self.regularizer.set_regularization_penalty(regularization_penalty)
    
    def set_regularization_method(self, regularization_method):
        if not self.fixed_size:
            self.regularizer.set_regularization_method(regularization_method)
    
    def get_param_string():
        param_string = ""
        weights_with_bias = tf.concat([self.W, tf.reshape(self.b, (1, -1))], axis=0)
        max_parameters = tf.math.reduce_max(tf.abs(weights_with_bias), axis=0).numpy()
        magnitudes = np.floor(np.log10(max_parameters))
        for m in magnitudes:
            if m > 0:
                m = 0
            param_string += str(int(-m))
        return param_string


class Conv2D(DASLayer):
    def __init__(self, filters, filter_size, activation, strides=(1, 1), 
                 padding='SAME', kernel_initializer='glorot_uniform',
                 bias_initializer='zeros', input_shape=None, fixed_size=False,
                 regularizer=None):
        super().__init__(input_shape)
    
        self.filters = filters
        self.filter_size = filter_size
        self.activation = activation
        self.strides = strides
        self.padding = padding
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        self.fixed_size = fixed_size
        
        self.A = tf.keras.activations.get(activation)
        self.F_init = tf.keras.initializers.get(kernel_initializer)
        self.b_init = tf.keras.initializers.get(bias_initializer)
        if regularizer is not None:
            self.regularizer = regularizer
        else:
            self.regularizer = Regularizer()            
        
    # def copy(self):
    #     input_shape = self._input_shape
    #     regularizer_copy = self.regularizer.copy()
    #     layer_copy = Conv2D(self.filters, self.filter_size, self.activation, self.strides, 
    #                         self.padding, self.kernel_initializer, self.bias_initializer, 
    #                         input_shape, self.fixed_size, regularizer_copy)
    #     return layer_copy
    
    def build(self, input_shape):
        input_filters = input_shape[-1]

        self.F = tf.Variable(
            name='F',
            initial_value=self.F_init(
                shape=(self.filter_size[0], self.filter_size[1], input_filters, self.filters), dtype=dtype
            ),
            trainable=True)
        
        self.b = tf.Variable(
            name='b',
            initial_value=self.b_init(shape=(self.filters,), dtype=dtype),
            trainable=True)

        self.add_regularizer_loss()
    
    def call(self, inputs, training=None):
        y = tf.nn.conv2d(inputs, self.F, strides=self.strides, padding=self.padding)
        y = tf.nn.bias_add(y, self.b)
        y = self.A(y)
        return y
    
    def add_regularizer_loss(self):
        self.add_loss(lambda: self.regularizer(tf.concat([tf.reshape(self.F, (-1, self.F.shape[-1])), tf.reshape(self.b, (1, -1))], axis=0)))
    
    def get_size(self):
        return self.F.shape[-2], self.F.shape[-1]
    
    def prune(self, threshold, active_input_units_indices):
        # Remove connections from pruned units in previous layer
        new_F = tf.gather(self.F.value(), active_input_units_indices, axis=-2)

        if self.fixed_size:
            active_output_filters_indices = list(range(new_F.shape[-1]))
        else:
            # Prune units in this layer
            F_reduced_max = tf.reshape(tf.math.reduce_max(tf.abs(new_F), axis=(0, 1, 2)), (1, -1))
            F_reduced_max_with_biases = tf.concat([F_reduced_max, tf.reshape(self.b.value(), (1, -1))], axis=0)
            filters_are_active = tf.math.reduce_max(tf.abs(F_reduced_max_with_biases), axis=0) >= threshold
            active_output_filters_indices = tf.reshape(tf.where(filters_are_active), (-1,))
            
            new_F = tf.gather(new_F, active_output_filters_indices, axis=-1)
            new_b = tf.gather(self.b.value(), active_output_filters_indices, axis=0)

            self.b = tf.Variable(name='b', initial_value=new_b, trainable=True)

        self.F = tf.Variable(name='F', initial_value=new_F, trainable=True)

        self.regularizer.prune()
        return active_output_filters_indices

    def grow(self, n_new_input_units, percentage, min_new_units, scaling_factor):
        if n_new_input_units > 0:
            # Add connections to grown units in previous layer
            F_growth = self.F_init(shape=(self.F.shape[0], self.F.shape[1], self.F.shape[2] + n_new_input_units, self.F.shape[3]), dtype=dtype)[:, :, -n_new_input_units:, :] * scaling_factor  # TODO is it better to be multiplying here by scaling_factor? It does help with not increasing the max weights of existing neurons when new neurons are added.
            new_F = tf.concat([self.F.value(), F_growth], axis=-2)
        else:
            new_F = self.F.value()

        if self.fixed_size:
            n_new_output_units = 0
        else:
            # Grow new units in this layer
            n_new_output_units = max(min_new_units, int(new_F.shape[-1] * percentage))
            if n_new_output_units > 0:
                F_growth = self.F_init(shape=(new_F.shape[0], new_F.shape[1], new_F.shape[2], new_F.shape[3] + n_new_output_units), dtype=dtype)[:, :, :, -n_new_output_units:] * scaling_factor
                b_growth = self.b_init(shape=(n_new_output_units,), dtype=dtype)  # TODO for all possible bias initializers to work properly, the whole bias vector should be initialized at once
                new_F = tf.concat([new_F, F_growth], axis=-1)
                new_b = tf.concat([self.b.value(), b_growth], axis=0)

                self.b = tf.Variable(name='b', initial_value=new_b, trainable=True)

        self.F = tf.Variable(name='F', initial_value=new_F, trainable=True)

        self.regularizer.grow(n_new_output_units)
        return n_new_output_units
    
    def mutate(self, mutation_strength):
        self.F.assign_add(tf.random.normal(self.F.shape, mean=0.0, stddev=mutation_strength))
        self.b.assign_add(tf.random.normal(self.b.shape, mean=0.0, stddev=mutation_strength))
    
    def set_regularization_penalty(self, regularization_penalty):
        if not self.fixed_size:
            self.regularizer.set_regularization_penalty(regularization_penalty)
    
    def set_regularization_method(self, regularization_method):
        if not self.fixed_size:
            self.regularizer.set_regularization_method(regularization_method)

    def get_param_string():
        param_string = ""
        # TODO
        return param_string


class Flatten(tf.keras.layers.Layer):
    def call(self, inputs, training=None):
        return tf.reshape(tf.transpose(inputs, perm=[0, 3, 1, 2]), (inputs.shape[0], -1))
    
    # def copy(self):
    #     return Flatten()


################################################################################
# MODELS
################################################################################


class Epoch:
    def __init__(self, grow, prune, regularization_penalty, regularization_method):
        self.grow = grow
        self.prune = prune
        self.regularization_penalty = regularization_penalty
        self.regularization_method = regularization_method
    
    def __str__(self):
        return f'{int(self.grow)}{int(self.prune)}{self.regularization_penalty}{self.regularization_method}'
    
    def __repr__(self):
        return self.__str__()


class DynamicEpoch(Epoch):
    def __init__(self, regularization_penalty, regularization_method):
        super().__init__(True, True, regularization_penalty, regularization_method)


class StaticEpoch(Epoch):
    def __init__(self, regularization_penalty, regularization_method):
        super().__init__(False, False, regularization_penalty, regularization_method)


class StaticEpochNoRegularization(StaticEpoch):
    def __init__(self):
        super().__init__(0., None)


class Schedule:
    def __init__(self, epochs):
        self.epochs = epochs

    def __iter__(self):
        return self.epochs.__iter__()
    
    def __len__(self):
        return len(self.epochs)
    
    def __str__(self):
        text = ''.join([str(epoch) for epoch in self.epochs])
        return hashlib.sha1(text.encode('utf-8')).hexdigest()[:10]
    
    def __repr__(self):
        return self.__str__()


class Sequential(tf.keras.Model):
    def __init__(self, layers):
        super().__init__()
        
        self.lrs = layers
        
    def call(self, inputs, training=None):
        x = inputs
        for layer in self.lrs:
            x = layer(x, training=training)
        return x
    
    def copy(self):
        copied_layers = list()
        for layer in self.lrs:
            if isinstance(layer, DASLayer):
                # layer_copy = layer.copy()
                layer_copy = copy.deepcopy(layer)
                layer_copy.add_regularizer_loss()
            else:
                layer_copy = copy.deepcopy(layer)
            copied_layers.append(layer_copy)
        
        model_copy = Sequential(copied_layers)
        return model_copy
    
    def get_layer_input_shape(self, target_layer):
        if target_layer._input_shape is not None:
            return target_layer._input_shape

        input = np.random.normal(size=(1,) + self.lrs[0]._input_shape)
        for layer in self.lrs:
            if layer is target_layer:
                return tuple(input.shape[1:])
            input = layer(input)
        raise Exception("Layer not found in the model.")

    def get_layer_output_shape(self, target_layer):
        input = np.random.normal(size=(1,) + self.lrs[0]._input_shape)
        for layer in self.lrs:
            output = layer(input)
            if layer is target_layer:
                return tuple(output.shape[1:])
            input = output
        raise Exception("Layer not found in the model.")
    
    def get_layer_sizes(self):
        """
        Returns the sizes of all layers in the model, including the input and output layer.
        """
        layer_sizes = list()
        first_layer = True
        for l in range(len(self.lrs)):
            layer = self.lrs[l]
            if isinstance(layer, DASLayer):
                layer_size = layer.get_size()
                if first_layer:
                    layer_sizes.append(layer_size[0])
                    first_layer = False
                layer_sizes.append(layer_size[1])
        return layer_sizes
    
    def get_hidden_layer_sizes(self):
        return self.get_layer_sizes()[1:-1]
    
    def get_regularization_penalty(self):
        #TODO improve
        return self.lrs[-2].regularizer.regularization_penalty
    
    def set_regularization_penalty(self, regularization_penalty):
        for layer in self.lrs:
            if isinstance(layer, DASLayer) and not layer.fixed_size:
                layer.set_regularization_penalty(regularization_penalty)
    
    def set_regularization_method(self, regularization_method):
        for layer in self.lrs:
            if isinstance(layer, DASLayer) and not layer.fixed_size:
                layer.set_regularization_method(regularization_method)

    def prune(self, params):
        input_shape = self.get_layer_input_shape(self.lrs[0])
        n_input_units = input_shape[-1]
        active_units_indices = list(range(n_input_units))

        last_custom_layer = None
        for layer in self.lrs:
            if isinstance(layer, DASLayer):
                if last_custom_layer is not None and type(last_custom_layer) != type(layer):
                    if type(last_custom_layer) == Conv2D and type(layer) == Dense:
                        convolutional_shape = self.get_layer_output_shape(last_custom_layer)
                        active_units_indices = self.convert_channel_indices_to_flattened_indices(active_units_indices, convolutional_shape)
                    else:
                        raise Exception("Incorrect order of custom layer types.")
                active_units_indices = layer.prune(params.pruning_threshold, active_units_indices)
                last_custom_layer = layer
    
    def grow(self, params):   
        n_new_units = 0

        last_custom_layer = None
        for layer in self.lrs:
            if isinstance(layer, DASLayer):
                if last_custom_layer is not None and type(last_custom_layer) != type(layer):
                    if type(last_custom_layer) == Conv2D and type(layer) == Dense:
                        convolutional_shape = self.get_layer_output_shape(last_custom_layer)
                        n_new_units = n_new_units * convolutional_shape[0] * convolutional_shape[1]
                    else:
                        raise Exception("Incorrect order of custom layer types.")
                n_new_units = layer.grow(n_new_units, params.growth_percentage, min_new_units=params.min_new_neurons, scaling_factor=params.pruning_threshold)
                last_custom_layer = layer
    
    def mutate(self, mutation_strength):
        for layer in self.lrs:
            if isinstance(layer, DASLayer):
                layer.mutate(mutation_strength)
    
    @staticmethod
    def convert_channel_indices_to_flattened_indices(channel_indices, convolutional_shape):
        dense_indices = list()
        units_per_channel = convolutional_shape[0] * convolutional_shape[1]
        for channel_index in channel_indices:
            for iter in range(units_per_channel):
                dense_indices.append(channel_index * units_per_channel + iter)
        return dense_indices
    
    def print_neurons(self):
        for layer in self.lrs[:-1]:
            print(layer.get_param_string())
    
    def evaluate(self, params, summed_training_loss, summed_training_metric):
        # Calculate training loss and metric
        if summed_training_loss is not None:
            loss = summed_training_loss / params.x.shape[0]
        else:
            loss = None
        
        if summed_training_metric is not None:
            metric = summed_training_metric / params.x.shape[0]
        else:
            metric = None
        
        # Calculate val loss and metric
        summed_val_loss = 0
        summed_val_metric = 0
        n_val_instances = 0
        
        for step, (x_batch, y_batch) in enumerate(params.val_dataset):
            # y_pred = tf.reshape(self(x_batch, training=False), y_batch.shape)
            y_pred = self(x_batch, training=False)
            summed_val_loss += tf.reduce_sum(params.loss_fn(y_batch, y_pred))
            summed_val_metric += float(tf.reduce_sum(params.metric_fn(y_batch, y_pred)))
            n_val_instances += x_batch.shape[0]
        
        val_loss = summed_val_loss / n_val_instances
        val_metric = summed_val_metric / n_val_instances

        return loss, metric, val_loss, val_metric

    def list_params(self):
        trainable_count = np.sum([K.count_params(w) for w in self.trainable_weights])
        non_trainable_count = np.sum([K.count_params(w) for w in self.non_trainable_weights])
        total_count = trainable_count + non_trainable_count

        print('Total params: {:,}'.format(total_count))
        print('Trainable params: {:,}'.format(trainable_count))
        print('Non-trainable params: {:,}'.format(non_trainable_count))

        return total_count, trainable_count, non_trainable_count
    
    def print_epoch_statistics(self, params, summed_training_loss, summed_training_metric, message=None, require_result=False):
        if not params.verbose:
            if require_result:
                return self.evaluate(params, summed_training_loss, summed_training_metric)
            else:
                return
        
        loss, metric, val_loss, val_metric = self.evaluate(params, summed_training_loss, summed_training_metric)  

        if message is not None:
            print(message)
        
        print(f"loss: {loss} - metric: {metric} - val_loss: {val_loss} - val_metric: {val_metric} - penalty: {self.get_regularization_penalty()}")
        hidden_layer_sizes = self.get_hidden_layer_sizes()
        print(f"hidden layer sizes: {hidden_layer_sizes}, total units: {sum(hidden_layer_sizes)}")
        if params.print_neurons:
            self.print_neurons()
        
        if require_result:
            return loss, metric, val_loss, val_metric
    
    def update_history(self, params, loss, metric, val_loss, val_metric):
        params.history['loss'].append(float(loss))
        params.history['metric'].append(float(metric))
        params.history['val_loss'].append(float(val_loss))
        params.history['val_metric'].append(float(val_metric))
        params.history['hidden_layer_sizes'].append(self.get_hidden_layer_sizes())
    
    @staticmethod
    def prepare_datasets(x, y, batch_size, validation_data):
        train_dataset = tf.data.Dataset.from_tensor_slices((x, y))
        train_dataset = train_dataset.shuffle(buffer_size=20000).batch(batch_size)
        val_dataset = tf.data.Dataset.from_tensor_slices(validation_data).batch(batch_size)
        return train_dataset.prefetch(tf.data.AUTOTUNE), val_dataset.prefetch(tf.data.AUTOTUNE)
    
    def manage_dynamic_regularization(self, params, val_loss):
        if val_loss >= params.best_conditional_val_loss * params.stall_coefficient:
            # Training is currently in stall
            if not params.training_stalled:
                penalty = self.get_regularization_penalty() * params.regularization_penalty_multiplier
                print("Changing penalty...")
                # TODO this must be modified, penalty can differ for each layer
                self.set_regularization_penalty(penalty)
                params.training_stalled = True
        else:
            params.best_conditional_val_loss = val_loss
            params.training_stalled = False
    
    def grow_wrapper(self, params):
        dynamic_reqularization_active = params.regularization_penalty_multiplier != 1.
        if dynamic_reqularization_active:
            loss, metric, val_loss, val_metric = self.print_epoch_statistics(params, None, None, "Before growing:", require_result=True)
            self.manage_dynamic_regularization(params, val_loss)
        else:
            self.print_epoch_statistics(params, None, None, "Before growing:")

        self.grow(params)
        self.print_epoch_statistics(params, None, None, "After growing:")
    
    def prune_wrapper(self, params, summed_loss, summed_metric):
        loss, metric, _, _ = self.print_epoch_statistics(params, summed_loss, summed_metric, "Before pruning:", require_result=True)
        self.prune(params)
        _, _, val_loss, val_metric = self.print_epoch_statistics(params, None, None, "After pruning:", require_result=True)

        self.update_history(params, loss, metric, val_loss, val_metric)
    
    class ParameterContainer:
        def __init__(self, x, y, optimizer, batch_size, min_new_neurons, validation_data, pruning_threshold, regularization_penalty_multiplier, 
                     stall_coefficient, growth_percentage, mini_epochs_per_epoch, verbose, print_neurons, use_static_graph, loss_fn, metric_fn):
            self.x = x
            self.y = y
            self.optimizer = optimizer
            self.batch_size = batch_size
            self.min_new_neurons = min_new_neurons
            self.validation_data = validation_data
            self.pruning_threshold = pruning_threshold
            self.regularization_penalty_multiplier = regularization_penalty_multiplier
            self.stall_coefficient = stall_coefficient
            self.growth_percentage = growth_percentage
            self.mini_epochs_per_epoch = mini_epochs_per_epoch
            self.verbose = verbose
            self.print_neurons = print_neurons
            self.use_static_graph = use_static_graph
            self.loss_fn = loss_fn
            self.metric_fn = metric_fn

            self.train_dataset, self.val_dataset = Sequential.prepare_datasets(x, y, batch_size, validation_data)
            self.history = self.prepare_history()

            self.best_conditional_val_loss = np.inf
            self.training_stalled = False
        
        @staticmethod
        def prepare_history():
            history = {
                'loss': list(),
                'metric': list(),
                'val_loss': list(),
                'val_metric': list(),
                'hidden_layer_sizes': list(),
            }
            return history
    
    def fit_single_step(self, params, x_batch, y_batch):
        with tf.GradientTape() as tape:
            # y_pred = tf.reshape(self(x_batch, training=True), y_batch.shape)
            y_pred = self(x_batch, training=True)
            raw_loss = params.loss_fn(y_batch, y_pred)
            loss_value = tf.reduce_mean(raw_loss)
            loss_value += sum(self.losses)  # Add losses registered by model.add_loss

            loss = tf.reduce_sum(raw_loss)
            metric = float(tf.reduce_sum(params.metric_fn(y_batch, y_pred)))

        grads = tape.gradient(loss_value, self.trainable_variables)
        params.optimizer.apply_gradients(zip(grads, self.trainable_variables))

        return loss, metric
    
    def fit_single_epoch(self, params):
        summed_loss = 0
        summed_metric = 0
        
        for mini_epoch in range(params.mini_epochs_per_epoch):
            summed_loss = 0
            summed_metric = 0

            if params.use_static_graph:
                fit_single_step_function = tf.function(self.fit_single_step)
            else:
                fit_single_step_function = self.fit_single_step
            for step, (x_batch, y_batch) in enumerate(params.train_dataset):
                loss, metric = fit_single_step_function(params, x_batch, y_batch)
                summed_loss += loss
                summed_metric += metric
        
        return summed_loss, summed_metric

    def fit(self, x, y, optimizer, schedule, batch_size, min_new_neurons, validation_data, pruning_threshold=0.001, regularization_penalty_multiplier=1., 
            stall_coefficient=1, growth_percentage=0.2, mini_epochs_per_epoch=1, verbose=True, print_neurons=False, use_static_graph=True, 
            loss_fn=tf.keras.losses.sparse_categorical_crossentropy, metric_fn=tf.keras.metrics.sparse_categorical_accuracy):
        params = self.ParameterContainer(x=x, y=y, optimizer=optimizer, batch_size=batch_size, min_new_neurons=min_new_neurons, validation_data=validation_data, 
                                         pruning_threshold=pruning_threshold, regularization_penalty_multiplier=regularization_penalty_multiplier, stall_coefficient=stall_coefficient, 
                                         growth_percentage=growth_percentage, mini_epochs_per_epoch=mini_epochs_per_epoch, verbose=verbose, print_neurons=print_neurons, 
                                         use_static_graph=use_static_graph, loss_fn=loss_fn, metric_fn=metric_fn)
        self.build(x.shape)  # Necessary when verbose == False

        for epoch_no, epoch in enumerate(schedule):
            if verbose:
                print("##########################################################")
                print(f"Epoch {epoch_no + 1}/{len(schedule)}")
            
            self.set_regularization_penalty(epoch.regularization_penalty)
            self.set_regularization_method(epoch.regularization_method)

            if epoch.grow:
                self.grow_wrapper(params)
            
            summed_loss, summed_metric = self.fit_single_epoch(params)

            if epoch.prune:
                self.prune_wrapper(params, summed_loss, summed_metric)
            else:
                loss, metric, val_loss, val_metric = self.print_epoch_statistics(params, summed_loss, summed_metric, require_result=True)
                self.update_history(params, loss, metric, val_loss, val_metric)
        
        return params.history


################################################################################
# HELPER FUNCTIONS
################################################################################


def get_statistics_from_history(history):
    best_epoch_number = np.argmin(history['val_loss'])
    best_val_loss = history['val_loss'][best_epoch_number]
    best_val_metric = history['val_metric'][best_epoch_number]
    best_hidden_layer_sizes = history['hidden_layer_sizes'][best_epoch_number]
    return best_val_loss, best_val_metric, best_hidden_layer_sizes


def get_statistics_from_histories(histories):
    best_val_losses = list()
    best_val_metrics = list()
    all_best_hidden_layer_sizes = list()

    for history in histories:
        best_val_loss, best_val_metric, best_hidden_layer_sizes = get_statistics_from_history(history)
        best_val_losses.append(best_val_loss)
        best_val_metrics.append(best_val_metric)
        all_best_hidden_layer_sizes.append(best_hidden_layer_sizes)
    
    mean_best_val_loss = np.mean(best_val_losses)
    mean_best_val_metric = np.mean(best_val_metrics)
    mean_best_hidden_layer_sizes = [np.mean(layer) for layer in list(zip(*all_best_hidden_layer_sizes))]
    
    return mean_best_val_loss, mean_best_val_metric, mean_best_hidden_layer_sizes


def cross_validate(train_fn, x, y, n_splits, random_state=42, *args, **kwargs):
    from sklearn.model_selection import KFold

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    histories = list()
    for i, (train_index, test_index) in enumerate(kf.split(x)):
        xtrain, xtest = x[train_index], x[test_index]
        ytrain, ytest = y[train_index], y[test_index]

        history = train_fn(xtrain, ytrain, validation_data=(xtest, ytest), *args, **kwargs)
        histories.append(history)

        best_val_loss, best_val_metric, best_hidden_layer_sizes = get_statistics_from_history(history)
        print(f"Run {i} completed, best_val_loss: {best_val_loss}, best_val_metric: {best_val_metric}, best_hidden_layer_sizes: {best_hidden_layer_sizes}")

    mean_best_val_loss, mean_best_val_metric, mean_best_hidden_layer_sizes = get_statistics_from_histories(histories)
    print(f'mean_best_val_loss: {mean_best_val_loss}')
    print(f'mean_best_val_metric: {mean_best_val_metric}')
    print(f'mean_best_hidden_layer_sizes: {mean_best_hidden_layer_sizes}')

    return histories, mean_best_hidden_layer_sizes


def hyperparameter_search(train_fn, x, y, validation_data, *args, **kwargs):
    from itertools import product

    all_params = [*args] + list(kwargs.values())
    histories = list()

    best_overall_val_loss = np.inf
    best_overall_val_metric = None
    best_overall_combination = None

    for combination in product(*all_params):
        combination_args = combination[:len(args)]

        combination_kwargs_values = combination[len(args):]
        combination_kwargs = dict(zip(kwargs.keys(), combination_kwargs_values))

        history = train_fn(x, y, validation_data, *combination_args, **combination_kwargs)
        history['parameters'] = combination
        histories.append(history)

        best_val_loss, best_val_metric, best_hidden_layer_sizes = get_statistics_from_history(history)
        print(f"Run with parameters {combination} completed, best_val_loss: {best_val_loss}, best_val_metric: {best_val_metric}, best_hidden_layer_sizes: {best_hidden_layer_sizes}")

        if best_val_loss < best_overall_val_loss:
            best_overall_val_loss = best_val_loss
            best_overall_val_metric = best_val_metric
            best_overall_combination = combination
    
    print(f'Best overall combination: {best_overall_combination}, val_metric: {best_overall_val_metric}')

    return histories, best_overall_combination


def get_convolutional_model(x, layer_sizes, output_neurons=10):
    model = Sequential([
        Conv2D(layer_sizes[0], filter_size=(3, 3), activation='selu', strides=(1, 1), padding='SAME', kernel_initializer='lecun_normal', input_shape=x[0,:,:,:].shape),
        Conv2D(layer_sizes[1], filter_size=(3, 3), activation='selu', strides=(2, 2), padding='SAME', kernel_initializer='lecun_normal'),
        tf.keras.layers.Dropout(0.2),
        Conv2D(layer_sizes[2], filter_size=(3, 3), activation='selu', strides=(1, 1), padding='SAME', kernel_initializer='lecun_normal'),
        Conv2D(layer_sizes[3], filter_size=(3, 3), activation='selu', strides=(2, 2), padding='SAME', kernel_initializer='lecun_normal'),
        tf.keras.layers.Dropout(0.5),
        Flatten(),
        Dense(layer_sizes[4], activation='selu', kernel_initializer='lecun_normal'),
        Dense(output_neurons, activation='softmax', fixed_size=True),
    ])
    return model


def get_dense_model(x, layer_sizes):
    layers = list()
    
    layers.append(Dense(layer_sizes[0], activation='selu', kernel_initializer='lecun_normal', input_shape=x[0, :].shape))
    for layer_size in layer_sizes[1:]:
        layers.append(Dense(layer_size, activation='selu', kernel_initializer='lecun_normal'))
    layers.append(Dense(1, activation=None, kernel_initializer='lecun_normal', fixed_size=True))
    
    model = Sequential(layers)
    return model


def train_fn_conv(x, y, validation_data, learning_rate, schedule, layer_sizes, output_neurons=10, min_new_neurons=20, 
             growth_percentage=0.2, verbose=False, use_static_graph=True):
    batch_size = 128

    model = get_convolutional_model(x, layer_sizes, output_neurons)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    history = model.fit(x=x, y=y, optimizer=optimizer, schedule=schedule, batch_size=batch_size, min_new_neurons=min_new_neurons, 
                        validation_data=validation_data, growth_percentage=growth_percentage, verbose=verbose, use_static_graph=use_static_graph)
    
    return history


def squared_error(y_true, y_pred):
    return (y_true - y_pred) ** 2


def train_fn_dense(x, y, validation_data, learning_rate, schedule, layer_sizes, min_new_neurons=20, 
             growth_percentage=0.2, verbose=False, use_static_graph=True):
    batch_size = 128

    model = get_dense_model(x, layer_sizes)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    history = model.fit(x=x, y=y, optimizer=optimizer, schedule=schedule, batch_size=batch_size, min_new_neurons=min_new_neurons, 
                        validation_data=validation_data, growth_percentage=growth_percentage, verbose=verbose, use_static_graph=use_static_graph,
                        loss_fn=squared_error, metric_fn=squared_error)
    
    return history

# Evolution

In [4]:
class Individual:
    def __init__(self, genome, model, optimizer, history=None, age=0):
        self.genome = genome
        self.model = model
        self.optimizer = optimizer
        if history is not None:
            self.history = history
        else:
            self.history = Sequential.ParameterContainer.prepare_history()
        self.age = age
    
    def copy(self):
        genome_copy = self.genome.copy()
        model_copy = self.model.copy()
        optimizer_copy = copy.deepcopy(self.optimizer)
        history_copy = copy.deepcopy(self.history)
        individual_copy = Individual(genome_copy, model_copy, optimizer_copy, history_copy, self.age)
        return individual_copy
    
    def mutate(self, mutation_strength):
        self.model.mutate(mutation_strength)
        self.age = 0
    
    def correct(self):
        self.genome[0] = max(self.genome[0], 2.5)  # Regularization penalty
        self.genome[1] = np.clip(self.genome[1], 0.1, 1)  # Dataset sample size
    
    def get_fitness(self):
        return self.history['val_metric'][-1]
    
    def get_regularization_penalty(self):
        return 10. ** -self.genome[0]
    
    def get_dataset_sample_size(self):
        return self.genome[1]
    
    def get_hidden_layer_sizes(self):
        return self.history['hidden_layer_sizes'][-1]


def initialize_population(population_size, x, layer_sizes, output_neurons, learning_rate):
    population = list()
    for _ in range(population_size):
        genome = np.array([3, 0.1])
        model = get_convolutional_model(x, layer_sizes, output_neurons=output_neurons)
        model.build(x.shape)
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        individual = Individual(genome, model, optimizer)
        population.append(individual)
    return population


def get_best_individual(population):
    best_individual = None
    best_fitness = - np.inf
    for individual in population:
        fitness = individual.get_fitness()
        if fitness > best_fitness:
            best_individual = individual
            best_fitness = fitness
    return best_individual


def crossover(population, n_parents, strategy):
    novel_population = list()
    for individual in population:
        parents_selection = np.random.choice(list(range(len(population))), size=n_parents, replace=False)
        parent_genomes = [population[index].genome for index in parents_selection]
        offspring_genome = np.mean(np.vstack(parent_genomes), axis=0)
        offspring = individual.copy()
        offspring.genome = offspring_genome
        offspring.genome += np.random.normal(0, 1, offspring.genome.shape) * strategy
        novel_population.append(offspring)
    return population + novel_population


def correct(population):
    for individual in population:
        individual.correct()
    return population


# def mutation(population, mutation_strength):
#     new_population = list()
#     for individual in population:
#         individual_copy = individual.copy()
#         individual_copy.mutate(mutation_strength)
#         new_population.extend([individual, individual_copy])
#     return new_population


def extend_history(old_history, new_history):
    for key in old_history.keys():
        old_history[key].extend(new_history[key])


def training(population, x, y, validation_data, batch_size, min_new_neurons, growth_percentage, verbose, use_static_graph):
    for individual in population:
        model = individual.model
        optimizer = individual.optimizer
        # schedule = Schedule([StaticEpochNoRegularization()])
        schedule = Schedule([DynamicEpoch(individual.get_regularization_penalty(), 'weighted_l1')])
        # x_train_sample, y_train_sample = get_dataset_sample(x, y, individual.get_dataset_sample_size())
        x_train_sample, y_train_sample = get_dataset_sample(x, y, 0.1)
        # x_test_sample, y_test_sample = get_dataset_sample(validation_data[0], validation_data[1], individual.get_dataset_sample_size())
        x_test_sample, y_test_sample = get_dataset_sample(validation_data[0], validation_data[1], 0.1, seed=42)
        history = model.fit(x=x_train_sample, y=y_train_sample, optimizer=optimizer, schedule=schedule, batch_size=batch_size, 
                            min_new_neurons=min_new_neurons, validation_data=(x_test_sample, y_test_sample), growth_percentage=growth_percentage, 
                            verbose=verbose, use_static_graph=use_static_graph)
        extend_history(individual.history, history)
        individual.age += 1
    return population


def tournament_selection(population, population_size, tournament_size):
    new_population = list()

    while len(new_population) < population_size:
        selection = np.random.choice(list(range(len(population))), size=tournament_size, replace=False)
        best_individual = None
        best_fitness = - np.inf
        for individual_index in selection:
            individual = population[individual_index]
            fitness = individual.get_fitness()
            if fitness > best_fitness:
                best_individual = individual
                best_fitness = fitness
        new_population.append(best_individual.copy())

    return new_population


def measure_fitnesses(population):
    fitnesses = list()
    for individual in population:
        fitnesses.append(individual.get_fitness())
    return fitnesses


def interruptible(f):
    def function(*args, **kwargs):
        try:
            return f(*args, **kwargs)
        except KeyboardInterrupt:
            print("Interrupted by user.")
    return function


def print_generation_statistics(generation, population, duration):
    sorted_population = sorted(population, key=lambda x: x.get_fitness(), reverse=True)
    individuals = [(individual.age, round(individual.get_fitness(), 4), round(individual.genome[0], 1), round(individual.genome[1], 2), individual.get_hidden_layer_sizes()) for individual in sorted_population]
    print(f"Generation {generation}: {round(duration, 1)} s, {individuals}")


@interruptible
def evolution(x, y, validation_data, batch_size, layer_sizes, output_neurons, learning_rate, n_parents, strategy, population_size=10, n_generations=10, 
              tournament_size=3, elitism=True, min_new_neurons=20, growth_percentage=0.2):
    population = initialize_population(population_size, x, layer_sizes, output_neurons, learning_rate)
    best_individual = None
    fitnesses_history = list()
    for generation in range(n_generations):
        start_time = time.time()
        population = crossover(population, n_parents, strategy)
        population = correct(population)
        # population = mutation(population, mutation_strength)
        population = training(population, x, y, validation_data, batch_size, min_new_neurons, 
                              growth_percentage, verbose=False, use_static_graph=True)
        population = tournament_selection(population, population_size, tournament_size)
        if elitism:
            if best_individual is not None:
                population.append(best_individual)
            best_individual = get_best_individual(population).copy()
        fitnesses = measure_fitnesses(population)
        duration = time.time() - start_time
        print_generation_statistics(generation, population, duration)
        # fitnesses_history.append(fitnesses)

    # best_individual = get_best_individual(population)
    # fitness = best_individual.get_fitness()

    # return fitness, fitnesses_history

## Playground for standalone models

In [5]:
# fashion_mnist = get_fashion_mnist_dataset(fraction=0.1)
cifar10 = get_cifar_10_dataset()

In [52]:
model = get_convolutional_model(cifar10.X_train_norm, [20, 20, 20, 20, 20], output_neurons=10)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0004)

In [39]:
%%time
schedule = Schedule([DynamicEpoch(0.001, 'weighted_l1')] * 3)
history = model.fit(x=cifar10.X_train_norm, y=cifar10.y_train, optimizer=optimizer, schedule=schedule, batch_size=32, min_new_neurons=20, 
                    validation_data=(cifar10.X_test_norm, cifar10.y_test), growth_percentage=0.2, verbose=True, 
                    use_static_graph=True)

Build called
Build called
Build called
Build called
##########################################################
Epoch 1/3
Before growing:
loss: None - metric: None - val_loss: 3.0269274711608887 - val_metric: 0.09990300678952474 - penalty: 0.001
hidden layer sizes: [20, 20, 20, 20, 20], total units: 100
After growing:
loss: None - metric: None - val_loss: 3.0269272327423096 - val_metric: 0.09990300678952474 - penalty: 0.001
hidden layer sizes: [40, 40, 40, 40, 40], total units: 200
Before pruning:
loss: 2.3111469745635986 - metric: 0.2078595608472824 - val_loss: 1.9085042476654053 - val_metric: 0.30940834141610085 - penalty: 0.001
hidden layer sizes: [40, 40, 40, 40, 40], total units: 200
After pruning:
loss: None - metric: None - val_loss: 1.908506155014038 - val_metric: 0.30940834141610085 - penalty: 0.001
hidden layer sizes: [20, 20, 20, 20, 20], total units: 100
##########################################################
Epoch 2/3
Before growing:
loss: None - metric: None - val_loss:

## Experiments

In [52]:
evolution(x=cifar10.X_train_norm, y=cifar10.y_train, validation_data=(cifar10.X_test_norm, cifar10.y_test),
          batch_size=32, layer_sizes=[20, 20, 20, 20, 20], output_neurons=10, learning_rate=0.0004, n_parents=5, mutation_strength=0.25, 
          population_size=10, n_generations=50, tournament_size=3)

[(1, 0.3259, 3), (1, 0.3288, 3), (1, 0.3453, 3.108894334009524), (1, 0.3579, 2.9342262850188043), (1, 0.3278, 2.8255903530138333), (1, 0.3366, 3.4943646588916417), (1, 0.3579, 2.9342262850188043), (1, 0.3366, 3.4943646588916417), (1, 0.3579, 2.9342262850188043), (1, 0.3288, 3)], 114.6 s
[(2, 0.3725, 2.9342262850188043), (2, 0.3676, 2.799887138119949), (2, 0.3783, 3.4943646588916417), (2, 0.3715, 2.9575499060170576), (2, 0.3705, 3), (2, 0.3948, 2.9342262850188043), (2, 0.3948, 2.9342262850188043), (2, 0.3705, 3), (2, 0.3725, 2.9342262850188043), (2, 0.3695, 3.549870512944016), (1, 0.3579, 2.9342262850188043)], 110.5 s
[(3, 0.3744, 2.872232056725549), (3, 0.3851, 2.9126201357924115), (3, 0.387, 3.029669968015206), (3, 0.4016, 2.9342262850188043), (3, 0.4016, 2.9342262850188043), (3, 0.3957, 2.9829794515015466), (3, 0.387, 3.029669968015206), (3, 0.3695, 2.9575499060170576), (3, 0.3851, 2.9126201357924115), (3, 0.4016, 2.9342262850188043), (2, 0.3948, 2.9342262850188043)], 132.7 s
[(4, 0.

In [53]:
evolution(x=cifar10.X_train_norm, y=cifar10.y_train, validation_data=(cifar10.X_test_norm, cifar10.y_test),
          batch_size=32, layer_sizes=[20, 20, 20, 20, 20], output_neurons=10, learning_rate=0.0004, n_parents=5, mutation_strength=0.5, 
          population_size=10, n_generations=50, tournament_size=3)

[(1, 0.3191, 3), (1, 0.3375, 2.8411915083977264), (1, 0.3191, 3), (1, 0.3249, 3), (1, 0.356, 2.6695199944380263), (1, 0.3337, 2.710941564524781), (1, 0.3375, 2.8411915083977264), (1, 0.3337, 3.471949620719608), (1, 0.3249, 3), (1, 0.356, 2.6695199944380263)], 122.2 s
[(2, 0.3851, 3.471949620719608), (2, 0.3666, 2.6420132063649913), (2, 0.3744, 3.294924807125864), (2, 0.3608, 2.797353499448219), (2, 0.3851, 3.471949620719608), (2, 0.3744, 3.294924807125864), (2, 0.3851, 3.471949620719608), (2, 0.3647, 2.710941564524781), (2, 0.3851, 3.471949620719608), (2, 0.3608, 2.797353499448219), (1, 0.356, 2.6695199944380263)], 129.8 s
[(3, 0.3686, 2.7929601049219506), (3, 0.4054, 3.3158819187250446), (3, 0.3967, 2.6420132063649913), (3, 0.4054, 3.3158819187250446), (3, 0.3754, 2.9826830627293273), (3, 0.3928, 3.471949620719608), (3, 0.4054, 3.3158819187250446), (3, 0.3928, 3.471949620719608), (3, 0.3754, 3.471949620719608), (3, 0.3841, 3.294924807125864), (2, 0.3851, 3.471949620719608)], 124.5 s
[

In [11]:
evolution(x=cifar10.X_train_norm, y=cifar10.y_train, validation_data=(cifar10.X_test_norm, cifar10.y_test),
          batch_size=32, layer_sizes=[20, 20, 20, 20, 20], output_neurons=10, learning_rate=0.0004, n_parents=5, mutation_strength=0.5, 
          population_size=10, n_generations=50, tournament_size=3)

Generation 0: 114.1 s, [(1, 0.3569, 2.4963026418315666, [40, 40, 40, 40, 40]), (1, 0.3569, 2.4963026418315666, [40, 40, 40, 40, 40]), (1, 0.356, 3.4262987166427443, [40, 40, 40, 40, 40]), (1, 0.356, 3.4262987166427443, [40, 40, 40, 40, 40]), (1, 0.356, 3.4262987166427443, [40, 40, 40, 40, 40]), (1, 0.3531, 3.1420714040802062, [40, 40, 40, 40, 40]), (1, 0.3453, 2.4988843363508497, [40, 40, 40, 40, 40]), (1, 0.3404, 3, [20, 20, 20, 20, 20]), (1, 0.3375, 3.0469875583662356, [40, 40, 40, 40, 40]), (1, 0.3366, 3, [20, 20, 20, 20, 20])]
Generation 1: 110.7 s, [(2, 0.3889, 3.4262987166427443, [60, 60, 60, 60, 60]), (2, 0.3889, 3.4262987166427443, [60, 60, 60, 60, 60]), (2, 0.3889, 3.4262987166427443, [60, 60, 60, 60, 60]), (2, 0.3889, 3.4262987166427443, [60, 60, 60, 60, 60]), (2, 0.3773, 3.4262987166427443, [60, 60, 60, 60, 60]), (2, 0.3763, 3, [40, 40, 40, 40, 40]), (2, 0.3734, 2.4963026418315666, [60, 60, 60, 60, 60]), (2, 0.3734, 2.4963026418315666, [60, 60, 60, 60, 60]), (2, 0.3608, 2.74

In [15]:
evolution(x=cifar10.X_train_norm, y=cifar10.y_train, validation_data=(cifar10.X_test_norm, cifar10.y_test),
          batch_size=32, layer_sizes=[20, 20, 20, 20, 20], output_neurons=10, learning_rate=0.0004, n_parents=5, mutation_strength=0.5, 
          population_size=10, n_generations=50, tournament_size=3)

Generation 0: 53.1 s, [(1, 0.3143, 3.2271322561109588, [20, 20, 20, 20, 20]), (1, 0.3104, 3.2296534408001984, [20, 20, 20, 20, 20]), (1, 0.3104, 3.4002216809995107, [20, 20, 20, 20, 20]), (1, 0.3055, 2.9638846470638054, [20, 20, 20, 20, 20]), (1, 0.3046, 2.81665912453963, [20, 20, 20, 20, 20]), (1, 0.2919, 2.884814783800078, [20, 20, 20, 20, 20]), (1, 0.2667, 2.5830946056579953, [20, 20, 20, 20, 20]), (1, 0.2367, 2.46487256174084, [20, 20, 20, 20, 20]), (1, 0.1862, 1.7808410753455155, [20, 20, 20, 20, 20]), (1, 0.1086, 1.4815408043116105, [20, 20, 20, 20, 20])]
Generation 1: 52.3 s, [(2, 0.3608, 3.332707853826636, [20, 20, 20, 20, 20]), (2, 0.3472, 3.265129093010425, [20, 20, 20, 20, 20]), (2, 0.3356, 3.200016116022627, [20, 20, 20, 20, 20]), (2, 0.3327, 3.4195007865782054, [20, 20, 20, 20, 20]), (2, 0.3278, 2.7791936827722936, [20, 20, 20, 20, 20]), (2, 0.322, 3.9191264397952983, [20, 20, 20, 32, 35]), (2, 0.3172, 3.0819722447553786, [20, 20, 20, 20, 20]), (2, 0.3046, 3.09932903868526

In [63]:
evolution(x=cifar10.X_train_norm, y=cifar10.y_train, validation_data=(cifar10.X_test_norm, cifar10.y_test),
          batch_size=32, layer_sizes=[20, 20, 20, 20, 20], output_neurons=10, learning_rate=0.0004, n_parents=5, mutation_strength=0.5, 
          population_size=10, n_generations=50, tournament_size=3)

Generation 0: 118.7 s, [(1, 0.3298, 3, [20, 20, 20, 20, 20]), (1, 0.3269, 3.154565372812397, [20, 20, 20, 20, 20]), (1, 0.3259, 3, [20, 20, 20, 20, 20]), (1, 0.3259, 3, [20, 20, 20, 20, 20]), (1, 0.3201, 2.691522902774604, [20, 20, 20, 20, 20]), (1, 0.3191, 3.55763229302829, [20, 20, 20, 20, 20]), (1, 0.3191, 3.55763229302829, [20, 20, 20, 20, 20]), (1, 0.3152, 3, [20, 20, 20, 20, 20]), (1, 0.3152, 3, [20, 20, 20, 20, 20]), (1, 0.3094, 3.506301457088, [20, 20, 20, 20, 20])]
Generation 1: 134.2 s, [(2, 0.3773, 3.767503190943379, [21, 23, 20, 20, 30]), (2, 0.3773, 3.767503190943379, [21, 23, 20, 20, 30]), (2, 0.3754, 3.722830530857286, [20, 21, 20, 20, 20]), (2, 0.3754, 3.722830530857286, [20, 21, 20, 20, 20]), (2, 0.3618, 4.1395795306043945, [24, 37, 27, 40, 38]), (2, 0.3618, 4.1395795306043945, [24, 37, 27, 40, 38]), (2, 0.3598, 3, [20, 20, 20, 20, 20]), (2, 0.3531, 3.55763229302829, [20, 20, 20, 20, 20]), (2, 0.3482, 3, [20, 20, 20, 20, 20]), (2, 0.3463, 3.506301457088, [20, 20, 20, 2

In [ ]:
evolution(x=cifar10.X_train_norm, y=cifar10.y_train, validation_data=(cifar10.X_test_norm, cifar10.y_test),
          batch_size=32, layer_sizes=[20, 20, 20, 20, 20], output_neurons=10, learning_rate=0.0004, n_parents=5, strategy=[0.5, 0.05], 
          population_size=10, n_generations=50, tournament_size=3)

2022-03-21 22:43:04.038726: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-21 22:43:04.572366: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-21 22:43:04.573003: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-21 22:43:04.573442: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

Generation 0: 130.2 s, [(1, 0.3366, 3.0, 0.1, [20, 20, 20, 20, 20]), (1, 0.3366, 3.0, 0.1, [20, 20, 20, 20, 20]), (1, 0.3366, 3.0, 0.1, [20, 20, 20, 20, 20]), (1, 0.3346, 2.9, 0.1, [20, 20, 20, 20, 20]), (1, 0.3269, 2.8, 0.1, [20, 20, 20, 20, 20]), (1, 0.3269, 2.8, 0.1, [20, 20, 20, 20, 20]), (1, 0.3269, 2.8, 0.1, [20, 20, 20, 20, 20]), (1, 0.324, 3.0, 0.1, [20, 20, 20, 20, 20]), (1, 0.3201, 2.9, 0.1, [20, 20, 20, 20, 20]), (1, 0.3104, 3.0, 0.1, [20, 20, 20, 20, 20])]
Generation 1: 151.9 s, [(2, 0.3831, 3.9, 0.19, [20, 21, 20, 25, 29]), (2, 0.3666, 3.4, 0.1, [20, 20, 20, 20, 20]), (2, 0.3453, 3.0, 0.1, [20, 20, 20, 20, 20]), (2, 0.3453, 3.0, 0.1, [20, 20, 20, 20, 20]), (2, 0.3375, 3.0, 0.1, [20, 20, 20, 20, 20]), (2, 0.3375, 3.0, 0.1, [20, 20, 20, 20, 20]), (2, 0.3375, 3.0, 0.1, [20, 20, 20, 20, 20]), (2, 0.3366, 3.0, 0.1, [20, 20, 20, 20, 20]), (1, 0.3366, 3.0, 0.1, [20, 20, 20, 20, 20]), (2, 0.322, 2.8, 0.1, [20, 20, 20, 20, 20]), (2, 0.322, 2.8, 0.1, [20, 20, 20, 20, 20])]
Generatio

Exception ignored in: <function ScopedTFGraph.__del__ at 0x7fd323e795e0>
Traceback (most recent call last):
  File "/home/cahlivoj/self-scaling-nets/.venv/lib/python3.8/site-packages/tensorflow/python/framework/c_api_util.py", line 54, in __del__
    self.deleter(self.graph)
KeyboardInterrupt: 
